# Precipitaciones
## datos de precipitaciones en la red de estaciones de Madrid

In [ ]:
%%html
<!-- Google tag (gtag.js) --> 

<script async src="https://www.googletagmanager.com/gtag/js?id=G-BCBM9LRTT2">
</script> 
<script> 
  window.dataLayer = window.dataLayer || []; 
  function gtag(){
    dataLayer.push(arguments);
    } 
  gtag('js', new Date()); 
  gtag('config', 'G-BCBM9LRTT2'); 
</script>

In [ ]:
%%html

<a href="index.html">General</a>

In [ ]:
import duckdb
import plotly.express as px

In [ ]:
%%capture
conn = duckdb.connect('/home/jcaubin/duck_test.db') 

%load_ext sql
%sql conn --alias duckdb

%config SqlMagic.displaylimit = 5 #conf de jupysql limite de filas a mostrar

In [ ]:
%%sql --save precip_total_estacion


pte2<<
select ano, mes, dia, make_date(ano, cast(mes as int), dia) fecha , estacion,estacion_desc, sum(valor) total
from v_CALAIR 
where magnitud = 89 --precipitacion 
and validez = 'V'
--and estacion_desc in ['Casa de Campo'] --, 'Juan Carlos I', 'El Pardo','Peñagrande', 'Plaza Elíptica']
and date_diff ('day',TS, current_date()) < (30)
group by all
order by all


In [ ]:
%%sql 
pm<<
select  fecha 
,avg(total) precip_media, max(total) precip_max, min(total) precip_min, median(total) precipt_mediana
, stddev(total) desviacion, count(total) n
from precip_total_estacion
group by all
order by all


In [ ]:
#grafico de precipitaciones
import plotly.express as px

df = pm.DataFrame()
fig = px.bar(df, x="fecha", y="precip_media", title = 'Precipitación media')
fig.show()

In [ ]:
#boxplot
df = pte2.DataFrame()

fig = px.box(df, x= 'fecha', y = 'total', title = 'Distribución precipitaciones', hover_name='ESTACION_DESC')
fig.show()


In [ ]:
%sql --close duckdb